In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import warnings
import pandas as pd

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


lista_produtos = pd.read_excel(r"base_produtos.xlsx")

# Abrir ChromeDriver em segundo plano
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--use-fake-ui-for-media-stream")
options.add_argument("--js-flags=--harmony")
options.add_argument('--disable-logging')
options.add_argument('log-level=0')
options.add_argument('log-level=1')
options.add_argument('log-level=2')
options.add_argument('log-level=3')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.experimental_options["prefs"] = chrome_prefs
navegador = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


def pesquisa_google(navegador, produto, preco_min, preco_max, termos):    
    lista_ofertas = []
    navegador.get('https://www.google.com/')
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(produto, Keys.ENTER)

    navegador.find_element(By.LINK_TEXT, 'Shopping').click()

    resultados = navegador.find_elements(By.CLASS_NAME, 'i0X6df')

    for resultado in resultados:
        nome  = resultado.find_element(By.CLASS_NAME,'tAxDx').text
        preco  = resultado.find_element(By.CLASS_NAME,'a8Pemb').text
        link = resultado.find_element(By.TAG_NAME,'a').get_attribute('href')

        if nome != '' and preco != '' and link != '':
            status = True
        for tb in produto.split(' '):
            if tb.lower() not in nome.lower():
                status = False
                
        for tb in str(termos).split(' '):
            if tb.lower() in nome.lower():
                status = False



        if status == True and preco_max > float(preco[3:].replace('.','').replace(',','.')) > preco_min:                
            #print(f'Nome: {nome}, Preço: {preco} e Link: {link}')
            dados = f'{nome},;,{preco},;,{link},;,google_shopping'
            if dados not in lista_ofertas:
                lista_ofertas.append(dados)
    return lista_ofertas


def pesquisa_buscape(navegador, produto, preco_min, preco_max, termos):
    lista_ofertas = []
    navegador.get('https://www.buscape.com.br/')

    navegador.find_element(By.TAG_NAME, 'input').send_keys(produto, Keys.ENTER)
    time.sleep(5)

    resultados = navegador.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')

    for resultado in resultados:
        nome   = resultado.find_element(By.TAG_NAME,'h2').text
        preco  = resultado.find_element(By.TAG_NAME,'p').text
        link   = resultado.get_attribute('href')



        if nome != '' and preco != '' and link != '':
            status = True
        for tb in produto[0].split(' '):
            if tb.lower() not in nome.lower():
                status = False

        for tb in str(termos).split(' '):
            if tb.lower() in nome.lower():
                status = False



        if status == True and preco_max > float(preco[3:].replace('.','').replace(',','.')) > preco_min:                
            #print(f'Nome: {nome}, Preço: {preco} e Link: {link}')
            dados = f'{nome},;,{preco},;,{link},;,buscape'
            if dados not in lista_ofertas:

                lista_ofertas.append(dados)
    return lista_ofertas


listas = []
for tb in lista_produtos.values:
    lista = pesquisa_google(navegador, tb[0], tb[1], tb[2], tb[3])
    lista1 = pesquisa_buscape(navegador, tb[0], tb[1], tb[2], tb[3])
    listas += lista + lista1
    
navegador.quit()

tabela = []
for tb in listas:
    tabela.append(tb.split(',;,'))
    
tabela = pd.DataFrame(tabela,columns=['Nome', 'Preco', 'Link', 'Site'])
tabela.to_excel('Pesquisa_Preco.xlsx', index = False)

